# Level 2 - Week 8 - 01 Endpoint Map and Boundaries

**Estimated time:** 60-90 minutes

## Learning Objectives

- List endpoints by role
- Clarify UI vs API responsibilities
- Keep /search for debugging


## Overview

Backend is the source of truth; UI is a client.
Map endpoints to user, admin, and system roles.

## Practice Steps

- Create an endpoint map.
- Tag each endpoint with owner.


### Sample code

Endpoint map template.


In [ ]:
endpoint_map = {
    'user': ['/chat', '/search'],
    'admin': ['/ingest', '/eval'],
    'system': ['/health', '/ready'],
}

print(endpoint_map)


### Student fill-in

Add notes on which layer owns which logic.


In [ ]:
# TODO: add ownership notes for each endpoint


## Self-check

- Is retrieval logic only in the backend?
- Is /search still available for debugging?


Legacy practice content from practice.ipynb

# Level 2 — Week 8 Practice: Productize RAG + Agents

**Estimated time:** 60–90 minutes

## Learning Objectives

- Define a minimal service checklist
- Ensure demo readiness (success + failure cases)
- Capture evidence for quality
- Align with 12-factor principles


Legacy practice content from practice.ipynb

## Overview

Productization means repeatable demos and evidence. You will build a compact
checklist and fill in a runbook.

You will:

1. Define a demo checklist.
2. Add a small runbook for setup.
3. Record evidence for success and failure cases.

## Practice Steps

- Fill in the checklist below.
- Add commands for running the service.
- Capture sample outputs.


Legacy practice content from practice.ipynb

### Task 8.1: Demo checklist

Fill in the checklist with concrete steps and outputs.


In [ ]:
# Legacy practice content
DEMO_CHECKLIST = """
- One-command run:
- One successful query:
- One failure-case query:
- Show citations:
- Show evaluation evidence:
- Capture logs:
"""

print(DEMO_CHECKLIST)


Legacy practice content from practice.ipynb

## Self-check

- Is the demo repeatable?
- Do you have evidence for success/failure?
- Are setup steps minimal?
